In [1]:
!pip install tensorrt

  Preparing metadata (setup.py) ... done
  Created wheel for tensorrt: filename=tensorrt-8.6.1.post1-py2.py3-none-any.whl size=17281 sha256=27239aa8760e689da6a90df783056550499b5d4c84c3d3a3e9b571735c3f0dfd
  Stored in directory: /root/.cache/pip/wheels/f4/c8/0e/b79b08e45752491b9acfdbd69e8a609e8b2ed7640dda5a3e59
Successfully built tensorrt


In [5]:
import tensorrt as trt

onnx_file_name = '/content/drive/MyDrive/inference/model/onnex/model.onnx'
tensorrt_file_name = '/content/drive/MyDrive/inference/model/tensorrt/model.plan'
TRT_LOGGER = trt.Logger(trt.Logger.WARNING)
EXPLICIT_BATCH = 1 << (int)(trt.NetworkDefinitionCreationFlag.EXPLICIT_BATCH)

trt.init_libnvinfer_plugins(TRT_LOGGER, "")

builder = trt.Builder(TRT_LOGGER)
network = builder.create_network(EXPLICIT_BATCH)
parser = trt.OnnxParser(network, TRT_LOGGER)

config = builder.create_builder_config()
config.max_workspace_size = 1 << 30

config.set_flag(trt.BuilderFlag.FP16)

with open(onnx_file_name, 'rb') as model:
    if not parser.parse(model.read()):
        for error in range(parser.num_errors):
            print (parser.get_error(error))

max_length = 300
profile = builder.create_optimization_profile()
profile.set_shape('input_ids', (1, max_length), (1, max_length), (1, max_length))
profile.set_shape('attention_mask', (1, max_length), (1, max_length), (1, max_length))
profile.set_shape('token_type_ids', (1, max_length), (1, max_length), (1, max_length))
config.add_optimization_profile(profile)

#engine = builder.build_cuda_engine(network)
plan = builder.build_serialized_network(network, config)
with trt.Runtime(TRT_LOGGER) as runtime:
    engine = runtime.deserialize_cuda_engine(plan)

buf = engine.serialize()
with open(tensorrt_file_name, 'wb') as f:
    f.write(buf)

TypeError: pybind11::init(): factory function returned nullptr